# 2017年11月15日20:46:13
# 利用DenseNet结构改进原CNN网络

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pickle
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from cumt_variant import densenet
sess=tf.InteractiveSession()
init=tf.global_variables_initializer()

In [ ]:
#cumt rgb 相片均值
cumt_picmean=[103.939, 116.779, 123.68]
cumt_data=np.load('cumt_data.pkl')
index=np.arange(cumt_data['data'].shape[0])
np.random.shuffle(index)
data=cumt_data['data'][index]
label=cumt_data['labels'][index]
del cumt_data
resize_img=[]
for i in range(data.shape[0]):
    img=data[i]+cumt_picmean
    img=cv2.resize(img.astype('uint8'),(128,128))
    resize_img.append(img.astype('uint8'))
    if i%10000==0:
        print (i)
data=None
resize_img=np.array(resize_img).astype('uint8')

In [ ]:
with tf.variable_scope('Placehloder'):
    X=tf.placeholder(dtype=tf.float32,shape=[None,128,128,3],name='X')
    Y=tf.placeholder(dtype=tf.float32,shape=[None,10],name='Y')
    bn_train=tf.placeholder(dtype=tf.bool,name='BN_FLAG')
with tf.name_scope('DenseNet'):
    model=densenet(Images=X,bn_istraining=bn_train,K=12,L=5,theta=0.5,output_class=10,sess=sess)
    y_score=model.y_score
with tf.name_scope('LOSS'):
    LOSS=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_score,labels=Y))
    tf.summary.scalar('loss',LOSS)
with tf.name_scope('TRAIN'):
    TRAIN=tf.train.AdamOptimizer(5e-4).minimize(LOSS)
with tf.name_scope('ACCURACY'):
    acc_count=tf.equal(tf.arg_max(y_score,1),tf.arg_max(Y,1))
    ACCURACY=tf.reduce_mean(tf.cast(acc_count,tf.float32))
    tf.summary.scalar('acc',ACCURACY)
bn_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)
log_name='densenet_cumt_1115'
writer_te=tf.summary.FileWriter(r'./mylog/'+log_name+r'/test',sess.graph)
writer_tr=tf.summary.FileWriter(r'./mylog/'+log_name+r'/train')
writer_val=tf.summary.FileWriter(r'./mylog/'+log_name+r'/val')
merge=tf.summary.merge_all()



In [ ]:
def get_trainable_variables_nums():
    sum_=0
    for s in list(tf.trainable_variables()):
#         print(s)
        s=str(s)
        shape_=s[s.index('(')+1:s.index(')')].split(',')
        s_=1
        #print(shape_)
        for i in shape_:
            if i =='':continue
            #print(i)
            s_*=int(i)
        sum_+=s_
    print('trainable variables nums:{},memory:{}Mb'.format(sum_,sum_/(1024*1024)))
get_trainable_variables_nums()

In [ ]:
data=resize_img

index=np.arange(data.shape[0])
np.random.shuffle(index)
# tr_index=index[:100]
# te_index=index[100:200]
tr_index=index[:int(data.shape[0]*0.9)]
te_index=index[int(data.shape[0]*0.9):]
tr_data,tr_label=data[tr_index],label[tr_index]
te_data,te_label=data[te_index],label[te_index]
del data
del label
del resize_img
print(tr_data.shape,te_data.shape)


val_d=np.load('valimg1k_1109.pkl')
val_data=[]
for v in range(val_d['data'].shape[0]):
    img_=cv2.resize(val_d['data'][v],(128,128))
    
    val_data.append(img_)
val_data=np.array(val_data)
val_label=val_d['label']
del val_d
print(val_data.shape)

In [ ]:
sess.run(tf.global_variables_initializer())
saver=tf.train.Saver()

In [ ]:
#开始正式训练
batchsize=104
best_acc=0
for i in range(1,1000*6*12):
    mask=np.random.choice(tr_data.shape[0],batchsize,replace=False)
    x_,y_=tr_data[mask]-cumt_picmean,tr_label[mask]
    feed_dict={X:x_,Y:y_,bn_train:True}
    sess.run([TRAIN,bn_ops],feed_dict=feed_dict)
    if i%30==0:
        feed_dict={X:x_,Y:y_,bn_train:False}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_tr.add_summary(m_,i)
        print('epoch:{},train loss:{},train accuracy:{}'.format(i,loss_,acc_))
    if i%20==0:
        mask=np.random.choice(te_data.shape[0],batchsize,replace=False)
        x_,y_=te_data[mask]-cumt_picmean,te_label[mask]
        feed_dict={X:x_,Y:y_,bn_train:False}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_te.add_summary(m_,i)
        print('--epoch:{},test loss:{},test accuracy:{}'.format(i,loss_,acc_))
    if i%10==0:
        mask=np.random.choice(val_data.shape[0],128,replace=False)
        x_,y_=val_data[mask]-cumt_picmean,val_label[mask]
        feed_dict={X:x_,Y:y_,bn_train:False}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        best_acc=max(best_acc,acc_)
        writer_val.add_summary(m_,i)
        print('@@epoch:{},val loss:{},val accuracy:{},best val acc:{}'.format(i,loss_,acc_,best_acc))
       

In [ ]:
saver.save(sess,'DenseNet/model_578',global_step=i)

## 使用CUMT训练好的模型进行迁移 学习

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pickle
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from cumt_variant import densenet
cumt_picmean=[103.939, 116.779, 123.68]
sess=tf.InteractiveSession()
init=tf.global_variables_initializer()

In [2]:
output_class=25
log_name='densenet5X5_trans'
with tf.variable_scope('Placehloder'):
    X=tf.placeholder(dtype=tf.float32,shape=[None,128,128,3],name='X')
    Y=tf.placeholder(dtype=tf.float32,shape=[None,output_class],name='Y')
    bn_train=tf.placeholder(dtype=tf.bool,name='BN_FLAG')
    LR=tf.placeholder(dtype=tf.float32,name='lr')
with tf.name_scope('DenseNet'):
    model=densenet(Images=X,bn_istraining=bn_train,K=12,L=5,theta=0.5,output_class=output_class,sess=sess)
    y_score=model.y_score
with tf.name_scope('LOSS'):
    LOSS=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_score,labels=Y))
    tf.summary.scalar('loss',LOSS)
with tf.name_scope('TRAIN'):
    TRAIN=tf.train.AdamOptimizer(LR).minimize(LOSS)
with tf.name_scope('ACCURACY'):
    acc_count=tf.equal(tf.arg_max(y_score,1),tf.arg_max(Y,1))
    ACCURACY=tf.reduce_mean(tf.cast(acc_count,tf.float32))
    tf.summary.scalar('acc',ACCURACY)
bn_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)

writer_te=tf.summary.FileWriter(r'./mylog/'+log_name+r'/test')
writer_tr=tf.summary.FileWriter(r'./mylog/'+log_name+r'/train')
writer_val=tf.summary.FileWriter(r'./mylog/'+log_name+r'/val')
merge=tf.summary.merge_all()

Building DenseNet....
(?, 64, 64, 24)
(?, 64, 64, 84)
(?, 32, 32, 42)
(?, 32, 32, 102)
(?, 16, 16, 51)
(?, 16, 16, 111)
(?, 8, 8, 55)
(?, 8, 8, 115)
(?, 4, 4, 57)


In [3]:
# <tf.Variable 'Tail/conv_class/weight:0' shape=(1, 1, 57, 10) dtype=float32_ref>
# <tf.Variable 'Tail/conv_class/biases:0' shape=(10,) dtype=float32_ref>
# <tf.Variable 'Tail/conv_class/BN/beta:0' shape=(10,) dtype=float32_ref>
# <tf.Variable 'Tail/conv_class/BN/gamma:0' shape=(10,) dtype=float32_ref>
model.init_network(sess=sess,weight_addr='densenet_1115.pkl',
                   skip_layer=['Tail/conv_class/weight','Tail/conv_class/biases','Tail/conv_class/BN/beta','Tail/conv_class/BN/gamma'],
                      )

loading weight file:densenet_1115.pkl
skip layer:Tail/conv_class/biases
skip layer:Tail/conv_class/BN/beta
skip layer:Tail/conv_class/weight
skip layer:Tail/conv_class/BN/gamma
network init done!


In [ ]:
sess.run(tf.global_variables_initializer())

In [4]:
#  'img4X4_1006.pkl'   'img5X5_240.pkl'
# d=np.load('img4X4_1006.pkl')
d=np.load('img5X5_950.pkl')
index_=np.arange(d['data'].shape[0])
np.random.shuffle(index_)
tr_index=index_[:int(d['data'].shape[0]*0.8)]
te_index=index_[int(d['data'].shape[0]*0.8):]
tr_data,tr_label=d['data'][tr_index],d['label'][tr_index]
te_data,te_label=d['data'][te_index],d['label'][te_index]
print(tr_data.shape,te_data.shape)
del d

(762, 128, 128, 3) (191, 128, 128, 3)


In [5]:
#开始正式训练
batchsize=104
lr_=1e-3
for i in range(1,1000*6*12):
    mask=np.random.choice(tr_data.shape[0],batchsize,replace=False)
    x_,y_=tr_data[mask]-cumt_picmean,tr_label[mask]
    feed_dict={X:x_,Y:y_,bn_train:True,LR:lr_}
    sess.run([TRAIN,bn_ops],feed_dict=feed_dict)
    if i%10==0:
        feed_dict={X:x_,Y:y_,bn_train:False}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_tr.add_summary(m_,i)
        print('epoch:{},train loss:{},train accuracy:{}'.format(i,loss_,acc_))
    if i%20==0:
        mask=np.random.choice(te_data.shape[0],batchsize,replace=False)
        x_,y_=te_data[mask]-cumt_picmean,te_label[mask]
        feed_dict={X:x_,Y:y_,bn_train:False}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_te.add_summary(m_,i)
        print('--epoch:{},test loss:{},test accuracy:{}'.format(i,loss_,acc_))
    if i%1000==0:
        lr_=max(lr_*0.2,1e-5)
        print('$$$ lr change:{}'.format(lr_))

epoch:10,train loss:3.9097437858581543,train accuracy:0.01923076994717121
epoch:20,train loss:3.266908884048462,train accuracy:0.03846153989434242
--epoch:20,test loss:3.239640712738037,test accuracy:0.03846153989434242
epoch:30,train loss:3.248006820678711,train accuracy:0.01923076994717121
epoch:40,train loss:3.2426977157592773,train accuracy:0.03846153989434242
--epoch:40,test loss:3.265066146850586,test accuracy:0.02884615585207939
epoch:50,train loss:3.1774749755859375,train accuracy:0.07692307978868484
epoch:60,train loss:3.2136383056640625,train accuracy:0.02884615585207939
--epoch:60,test loss:3.2313990592956543,test accuracy:0.0
epoch:70,train loss:3.1690196990966797,train accuracy:0.02884615585207939
epoch:80,train loss:3.1648733615875244,train accuracy:0.07692307978868484
--epoch:80,test loss:3.1354827880859375,test accuracy:0.08653846383094788
epoch:90,train loss:3.146747350692749,train accuracy:0.10576924681663513
epoch:100,train loss:3.093513011932373,train accuracy:0.038

--epoch:760,test loss:0.5507946610450745,test accuracy:0.8750000596046448
epoch:770,train loss:0.44151705503463745,train accuracy:0.8557692766189575
epoch:780,train loss:0.39074766635894775,train accuracy:0.9038462042808533
--epoch:780,test loss:0.8246680498123169,test accuracy:0.7884616255760193
epoch:790,train loss:0.27957671880722046,train accuracy:0.9230769872665405
epoch:800,train loss:0.06539977341890335,train accuracy:1.0000001192092896
--epoch:800,test loss:0.1284262239933014,test accuracy:0.9903846383094788
epoch:810,train loss:0.44208571314811707,train accuracy:0.9038461446762085
epoch:820,train loss:0.03467787057161331,train accuracy:1.0
--epoch:820,test loss:0.20070669054985046,test accuracy:0.9711539149284363
epoch:830,train loss:0.06678452342748642,train accuracy:1.0
epoch:840,train loss:0.04074660688638687,train accuracy:1.0000001192092896
--epoch:840,test loss:0.14839491248130798,test accuracy:0.9615384936332703
epoch:850,train loss:0.0426672101020813,train accuracy:1.0

--epoch:1540,test loss:0.12441558390855789,test accuracy:0.9615384340286255
epoch:1550,train loss:0.013888044282793999,train accuracy:1.0
epoch:1560,train loss:0.014154157601296902,train accuracy:1.0
--epoch:1560,test loss:0.0583760105073452,test accuracy:0.9903846979141235
epoch:1570,train loss:0.014400150626897812,train accuracy:1.0
epoch:1580,train loss:0.017459798604249954,train accuracy:1.0000001192092896
--epoch:1580,test loss:0.07164329290390015,test accuracy:0.9903846383094788
epoch:1590,train loss:0.016939502209424973,train accuracy:1.0
epoch:1600,train loss:0.015654824674129486,train accuracy:1.0
--epoch:1600,test loss:0.1317906230688095,test accuracy:0.9711539149284363
epoch:1610,train loss:0.015237884595990181,train accuracy:1.0000001192092896
epoch:1620,train loss:0.013028129003942013,train accuracy:1.0
--epoch:1620,test loss:0.10587391257286072,test accuracy:0.9711538553237915
epoch:1630,train loss:0.012621833011507988,train accuracy:1.0
epoch:1640,train loss:0.0124058220

--epoch:2340,test loss:0.09750078618526459,test accuracy:0.9807692766189575
epoch:2350,train loss:0.011113162152469158,train accuracy:1.0
epoch:2360,train loss:0.011355224065482616,train accuracy:1.0
--epoch:2360,test loss:0.08764494210481644,test accuracy:0.9807692766189575
epoch:2370,train loss:0.010907415300607681,train accuracy:1.0
epoch:2380,train loss:0.011519338935613632,train accuracy:1.0
--epoch:2380,test loss:0.09434859454631805,test accuracy:0.9807692766189575
epoch:2390,train loss:0.010798927396535873,train accuracy:1.0
epoch:2400,train loss:0.010600087232887745,train accuracy:1.0
--epoch:2400,test loss:0.08924022316932678,test accuracy:0.9711538553237915
epoch:2410,train loss:0.011085277423262596,train accuracy:1.0
epoch:2420,train loss:0.010554701089859009,train accuracy:1.0
--epoch:2420,test loss:0.058887846767902374,test accuracy:0.990384578704834
epoch:2430,train loss:0.014502951875329018,train accuracy:1.0
epoch:2440,train loss:0.03695095330476761,train accuracy:1.000

epoch:3140,train loss:0.010315182618796825,train accuracy:1.0
--epoch:3140,test loss:0.07967951148748398,test accuracy:0.9807692766189575
epoch:3150,train loss:0.010156157426536083,train accuracy:1.0
epoch:3160,train loss:0.010015837848186493,train accuracy:1.0
--epoch:3160,test loss:0.0530465766787529,test accuracy:0.990384578704834
epoch:3170,train loss:0.009831340983510017,train accuracy:1.0
epoch:3180,train loss:0.009865636005997658,train accuracy:1.0
--epoch:3180,test loss:0.046358782798051834,test accuracy:0.9903846979141235
epoch:3190,train loss:0.009778892621397972,train accuracy:1.0
epoch:3200,train loss:0.010074364021420479,train accuracy:1.0
--epoch:3200,test loss:0.10641314834356308,test accuracy:0.9711538553237915
epoch:3210,train loss:0.009400301612913609,train accuracy:1.0
epoch:3220,train loss:0.009669244289398193,train accuracy:1.0000001192092896
--epoch:3220,test loss:0.06073817238211632,test accuracy:0.9807692766189575
epoch:3230,train loss:0.010054508224129677,train

...

--epoch:67300,test loss:0.1129884421825409,test accuracy:0.9615384936332703
epoch:67310,train loss:0.0018596118316054344,train accuracy:1.0000001192092896
epoch:67320,train loss:0.0018379861721768975,train accuracy:1.0000001192092896
--epoch:67320,test loss:0.11054740846157074,test accuracy:0.9615384936332703
epoch:67330,train loss:0.0017932093469426036,train accuracy:1.0
epoch:67340,train loss:0.0018009730847552419,train accuracy:1.0000001192092896
--epoch:67340,test loss:0.05721167474985123,test accuracy:0.9711538553237915
epoch:67350,train loss:0.0017368062399327755,train accuracy:1.0
epoch:67360,train loss:0.0018204095540568233,train accuracy:1.0
--epoch:67360,test loss:0.1149713471531868,test accuracy:0.9615384936332703
epoch:67370,train loss:0.001694581238552928,train accuracy:1.0
epoch:67380,train loss:0.0017371075227856636,train accuracy:1.0
--epoch:67380,test loss:0.060124196112155914,test accuracy:0.9807692170143127
epoch:67390,train loss:0.0017166177276521921,train accuracy:

--epoch:68080,test loss:0.08408623933792114,test accuracy:0.9711538553237915
epoch:68090,train loss:0.0016239869873970747,train accuracy:1.0
epoch:68100,train loss:0.0017167484620586038,train accuracy:1.0
--epoch:68100,test loss:0.08732172101736069,test accuracy:0.9711538553237915
epoch:68110,train loss:0.0016351697267964482,train accuracy:1.0
epoch:68120,train loss:0.001714245998300612,train accuracy:1.0
--epoch:68120,test loss:0.09209965914487839,test accuracy:0.9711538553237915
epoch:68130,train loss:0.0017324271611869335,train accuracy:1.0
epoch:68140,train loss:0.0017270720563828945,train accuracy:1.0
--epoch:68140,test loss:0.058240488171577454,test accuracy:0.9807692766189575
epoch:68150,train loss:0.0018353966297581792,train accuracy:1.0
epoch:68160,train loss:0.0018380769761279225,train accuracy:1.0
--epoch:68160,test loss:0.07919620722532272,test accuracy:0.9711538553237915
epoch:68170,train loss:0.0018294693436473608,train accuracy:1.0
epoch:68180,train loss:0.00184798077680

epoch:68870,train loss:0.00173270667437464,train accuracy:1.0
epoch:68880,train loss:0.0017442890675738454,train accuracy:1.0
--epoch:68880,test loss:0.07249433547258377,test accuracy:0.9711539149284363
epoch:68890,train loss:0.0016950276913121343,train accuracy:1.0
epoch:68900,train loss:0.0016871306579560041,train accuracy:1.0
--epoch:68900,test loss:0.030008234083652496,test accuracy:0.9903846383094788
epoch:68910,train loss:0.0016900720074772835,train accuracy:1.0
epoch:68920,train loss:0.0017448333092033863,train accuracy:1.0
--epoch:68920,test loss:0.0738665759563446,test accuracy:0.9711538553237915
epoch:68930,train loss:0.0017839923966675997,train accuracy:1.0000001192092896
epoch:68940,train loss:0.001856893883086741,train accuracy:1.0
--epoch:68940,test loss:0.0750177875161171,test accuracy:0.9807692766189575
epoch:68950,train loss:0.0018975051352754235,train accuracy:1.0000001192092896
epoch:68960,train loss:0.0018194369040429592,train accuracy:1.0000001192092896
--epoch:689

epoch:69650,train loss:0.001636626897379756,train accuracy:1.0
epoch:69660,train loss:0.0016697677783668041,train accuracy:1.0
--epoch:69660,test loss:0.02768329344689846,test accuracy:0.990384578704834
epoch:69670,train loss:0.0016929423436522484,train accuracy:1.0000001192092896
epoch:69680,train loss:0.0016955368919298053,train accuracy:1.0
--epoch:69680,test loss:0.09953197836875916,test accuracy:0.961538553237915
epoch:69690,train loss:0.0017047354485839605,train accuracy:1.0
epoch:69700,train loss:0.0017071316251531243,train accuracy:1.0
--epoch:69700,test loss:0.021952837705612183,test accuracy:0.990384578704834
epoch:69710,train loss:0.0017206883057951927,train accuracy:1.0
epoch:69720,train loss:0.0016859227325767279,train accuracy:1.0
--epoch:69720,test loss:0.0970548540353775,test accuracy:0.9711539149284363
epoch:69730,train loss:0.0017363785300403833,train accuracy:1.0
epoch:69740,train loss:0.0017352637369185686,train accuracy:1.0
--epoch:69740,test loss:0.092578813433647

KeyboardInterrupt: 

In [6]:
saver=tf.train.Saver()
saver.save(sess,'./model_save/Densenet_trans/model_991.ckpt',global_step=i)

'./model_save/Densenet_trans/model_991.ckpt-69823'

In [8]:
model.save_network_weight('dense4x4_trans991.pkl',sess)

extracting network weight to file:dense4x4_trans991.pkl
Head/conv1_3X3/weight
Head/conv1_3X3/biases
Head/conv1_3X3/BN/beta
Head/conv1_3X3/BN/gamma
Head/conv2_3X3/weight
Head/conv2_3X3/biases
Head/conv2_3X3/BN/beta
Head/conv2_3X3/BN/gamma
DenseConect/denseblock_1/dense_1/bottleneck_1X1/weight
DenseConect/denseblock_1/dense_1/bottleneck_1X1/biases
DenseConect/denseblock_1/dense_1/bottleneck_1X1/BN/beta
DenseConect/denseblock_1/dense_1/bottleneck_1X1/BN/gamma
DenseConect/denseblock_1/dense_1/composite_3X3/weight
DenseConect/denseblock_1/dense_1/composite_3X3/biases
DenseConect/denseblock_1/dense_1/composite_3X3/BN/beta
DenseConect/denseblock_1/dense_1/composite_3X3/BN/gamma
DenseConect/denseblock_1/dense_2/bottleneck_1X1/weight
DenseConect/denseblock_1/dense_2/bottleneck_1X1/biases
DenseConect/denseblock_1/dense_2/bottleneck_1X1/BN/beta
DenseConect/denseblock_1/dense_2/bottleneck_1X1/BN/gamma
DenseConect/denseblock_1/dense_2/composite_3X3/weight
DenseConect/denseblock_1/dense_2/composite_

DenseConect/denseblock_4/dense_3/bottleneck_1X1/weight
DenseConect/denseblock_4/dense_3/bottleneck_1X1/biases
DenseConect/denseblock_4/dense_3/bottleneck_1X1/BN/beta
DenseConect/denseblock_4/dense_3/bottleneck_1X1/BN/gamma
DenseConect/denseblock_4/dense_3/composite_3X3/weight
DenseConect/denseblock_4/dense_3/composite_3X3/biases
DenseConect/denseblock_4/dense_3/composite_3X3/BN/beta
DenseConect/denseblock_4/dense_3/composite_3X3/BN/gamma
DenseConect/denseblock_4/dense_4/bottleneck_1X1/weight
DenseConect/denseblock_4/dense_4/bottleneck_1X1/biases
DenseConect/denseblock_4/dense_4/bottleneck_1X1/BN/beta
DenseConect/denseblock_4/dense_4/bottleneck_1X1/BN/gamma
DenseConect/denseblock_4/dense_4/composite_3X3/weight
DenseConect/denseblock_4/dense_4/composite_3X3/biases
DenseConect/denseblock_4/dense_4/composite_3X3/BN/beta
DenseConect/denseblock_4/dense_4/composite_3X3/BN/gamma
DenseConect/denseblock_4/dense_5/bottleneck_1X1/weight
DenseConect/denseblock_4/dense_5/bottleneck_1X1/biases
DenseC

In [ ]:
%%timeit
mask=np.random.choice(10,1,replace=False)
x_,y_=te_data[mask]-cumt_picmean,te_label[mask]
feed_dict={X:x_,Y:y_,bn_train:False}
sess.run(y_score,feed_dict=feed_dict)

## 实测 5X5分类

In [9]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')  
def drawline(img_,line_w=1,line_color=(0,0,0),wandh_num=4):
        '''
        在图片上格子
        :param line_w: 线宽
        :param line_color: 线颜色
        :param wandh_num:  长宽线的数量
        :return:  无
        '''
        h,w=img_.shape[0],img_.shape[1]
        w_num,h_num=wandh_num,wandh_num
        h_,w_=h//h_num,w//w_num

        # 竖线 (w,h)
        for i in range(1,w_num):
            #print(i)
            cv2.line(img_,(w_*i,0),(w_*i,h),line_color,line_w)
        # 横线
        for i in range(1,h_num):
            cv2.line(img_,(0,h_*i),(w,h_*i),line_color,line_w)
        return img_

def drawblock(img,line_num,block_id=0,blockcolor=(46,218,255),blockwideth=5,show_rec='fill',rec_shape=None):
        '''
        选定九宫格，在这个格子上填充矩形表示选定这个格子
        :param img_: 图片
        :param block: 九宫格序号 0-15
        :param blockcolor: 矩形框颜色
        :param blockwideth: 框的宽度
        :return:
        '''
        h,w=img.shape[0],img.shape[1]
        w_line,h_line=line_num,line_num
        h_,w_=h//h_line,w//w_line
        cor_h=block_id//line_num
        cor_w=block_id%line_num
        sx,sy=cor_w*w_,cor_h*h_
        
        if show_rec=='fill':
        #将整个矩形填充为其他颜色
            img[sy:sy+h_,sx:sx+w_,:]=blockcolor
        elif show_rec=='rec':
            #显示矩形轮廓
            xe,ye,we,he=rec_shape
            cv2.rectangle(img,(xe,ye),(xe+we,ye+he),(255,0,0),10)
        elif show_rec=='dot':
            #标记一个小点
            roi_=img[sy:sy+h_,sx:sx+w_]
            cv2.circle(roi_,(roi_.shape[1]//2,roi_.shape[0]//2), 10, (255,128,120), -1)
        elif show_rec=='select':
            #hight light 矩形区域
            #img[sy:sy+h_,sx:sx+w_,0]=255#r
            img[sy:sy+h_,sx:sx+w_,1]=255#g
            #img[sy:sy+h_,sx:sx+w_,2]=255#b
        elif show_rec=='pick':
            #返回矩形框内容
            return img[sy:sy+h_,sx:sx+w_],(sx,sy,w_,h_)
        return img
def geteyeimg(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    faces = face_cascade.detectMultiScale(gray, 1.3,5) 
    if len(faces) !=1:
        print(len(faces))
        print('bad faces')
        return None
    for (x,y,w,h) in faces: 
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2) 
            roi_gray = gray[y:y+h, x:x+w] 
            roi_color = img[y:y+h, x:x+w] 
            #检测视频中脸部的眼睛，并用vector保存眼睛的坐标、大小（用矩形表示） 
            eyes = eye_cascade.detectMultiScale(roi_gray,scaleFactor=1.2, minNeighbors=7, minSize=(29, 29),
                                     flags=cv2.CASCADE_SCALE_IMAGE) 
            #眼睛检测 ,对于识别比较差的情况舍弃
            if len(eyes)!=2:
                print(len(eyes))
                print('bad eyes')
                return None
            if eyes[0][0]>eyes[1][0]:
                ex=eyes[1][0]
                W=eyes[0][0]-eyes[1][0]+eyes[0][2]
            else:
                ex=eyes[0][0]
                W=eyes[1][0]-eyes[0][0]+eyes[1][2]
            
            if eyes[0][1]>eyes[1][1]:
                ey=eyes[1][1]
                H=eyes[0][1]-eyes[1][1]+eyes[0][3]
            else:
                ey=eyes[0][1]
                H=eyes[1][1]-eyes[0][1]+eyes[1][3]
    
    return roi_color[ey+10:ey+H-2,ex-10:ex+W+10] if roi_color is not None else None

In [10]:
cap=cv2.VideoCapture(0)
_,frame=cap.read()
cap.release()

In [11]:
prob=tf.nn.softmax(y_score)

In [12]:
e=geteyeimg(frame)

In [13]:
e=cv2.resize(e,(128,128))[:,:,::-1].reshape((1,128,128,3))

In [14]:
#e=get_eyeimg(frame)
prob_=sess.run(prob,feed_dict={X:e-cumt_picmean,bn_train:False})

In [15]:
prob_

array([[ 0.00859234,  0.00745736,  0.00759149,  0.00818412,  0.00734552,
         0.0078813 ,  0.01121515,  0.00832109,  0.00946202,  0.0104944 ,
         0.00879968,  0.00738507,  0.01553816,  0.00748385,  0.00814339,
         0.00758778,  0.5859043 ,  0.20858185,  0.00734552,  0.00948442,
         0.00851893,  0.01254896,  0.00933308,  0.00887021,  0.0079301 ]], dtype=float32)

In [16]:
import time

In [17]:
fps=10
fsize_desk=(1920,1080)
# save video
video_d=cv2.VideoWriter('demo_desk.flv',cv2.VideoWriter_fourcc('F','L','V','1'),fps,fsize_desk)
cap=cv2.VideoCapture(0)
s_time=time.time()
tar_img=cv2.imread('test.jpg')
#设置window 为全屏
cv2.namedWindow('Capture001',cv2.WINDOW_NORMAL)
cv2.setWindowProperty('Capture001', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

s_time=time.time()
random_show=np.random.randint(0,16,1)[0]

#每N帧输出平均预测
sum_score=[]
sum_counter=0
block_id=0
ret_counter=0
right_frame_counter=0
while True:
    ret,fram=cap.read()
    if ret:
        ret_counter+=1
        #显示 内窥镜图像
        tar_img=cv2.imread('test.jpg')
        tar_img=drawline(tar_img,wandh_num=5)
        #每30帧换点
        if ret_counter%22==0:
            random_show=np.random.randint(0,16,1)[0]

        tar_img=drawblock(tar_img,line_num=5,block_id=random_show)
        cv2.imshow('Capture001',tar_img)
        e=geteyeimg(fram)
        if e is None:
            continue
        e=cv2.resize(e,(128,128))[:,:,::-1].reshape((1,128,128,3))
        y_guess=sess.run(prob,feed_dict={X:e-cumt_picmean,bn_train:False})
        sum_score.append(y_guess)
        #平均每 N 帧的预测分数
        if ret_counter%5==0:
            sum_score=np.array(sum_score)
            mean_=np.mean(sum_score,0)
            block_id=np.argmax(mean_)-1
            sum_score=[]
        if block_id<0:block_id=0
        elif block_id>25:block_id=25

        print(block_id)
        if block_id==random_show:
             right_frame_counter+=1
        tar_img=drawblock(tar_img,line_num=5,block_id=block_id,show_rec='dot')
        v_img=tar_img.copy()
        fram=cv2.resize(fram,(150,150))
        v_img[:150,-150:]=fram
        cv2.imshow('Capture001',tar_img)


#         print('{} sec per frame'.format(time.time()-s_time))
        s_time=time.time()
        video_d.write(v_img)
        if cv2.waitKey(1)&0xff==27:
            print('accuracy:{}'.format(right_frame_counter/ret_counter))
            print('out')
            break
    if (time.time()-s_time)>60*5:
        print('time out')
        break
video_d.release()
cap.release()
cv2.destroyAllWindows()


0
0
0
0
6
6
6
6
6
6
6
6
6
6
23
23
23
23
23
23
23
23
23
23
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
5
5
5
5
5
6
6
6
6
6
6
6
6
6
6
5
5
5
5
5
5
5
5
5
5
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
1
1
1
1
1
1
1
1
1
1
6
6
6
6
6
6
6
6
6
6
1
1
1
1
1
1
1
1
1
1
6
6
6
6
6
6
6
6
6
6
9
9
9
9
9
9
9
9
9
9
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
18
18
18
18
18
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
13
13
13
13
13
13
13
13
13
13
19
19
19
19
19
13
13
13
13
13
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
23
23
23
23


In [ ]:
video_d.release()
cap.release()
cv2.destroyAllWindows()
